# Detecting and mitigating age bias on credit decisions 

The goal of this tutorial is to introduce the basic functionality of AI Fairness 360 to an interested developer who may not have a background in bias detection and mitigation.

### Biases and Machine Learning
A machine learning model makes predictions of an outcome for a particular instance. (Given an instance of a loan application, predict if the applicant will repay the loan.) The model makes these predictions based on a training dataset, where many other instances (other loan applications) and actual outcomes (whether they repaid) are provided. Thus, a machine learning algorithm will attempt to find patterns, or generalizations, in the training dataset to use when a prediction for a new instance is needed. (For example, one pattern it might discover is "if a person has salary > USD 40K and has outstanding debt < USD 5, they will repay the loan".) In many domains this technique, called supervised machine learning, has worked very well.

However, sometimes the patterns that are found may not be desirable or may even be illegal. For example, a loan repay model may determine that age plays a significant role in the prediction of repayment because the training dataset happened to have better repayment for one age group than for another. This raises two problems: 1) the training dataset may not be representative of the true population of people of all age groups, and 2) even if it is representative, it is illegal to base any decision on a applicant's age, regardless of whether this is a good prediction based on historical data.

AI Fairness 360 is designed to help address this problem with _fairness metrics_ and _bias mitigators_.  Fairness metrics can be used to check for bias in machine learning workflows.  Bias mitigators can be used to overcome bias in the workflow to produce a more fair outcome. 

The loan scenario describes an intuitive example of illegal bias. However, not all undesirable bias in machine learning is illegal it may also exist in more subtle ways.  For example, a loan company may want a diverse portfolio of customers across all income levels, and thus, will deem it undesirable if they are making more loans to high income levels over low income levels.  Although this is not illegal or unethical, it is undesirable for the company's strategy.

As these two examples illustrate, a bias detection and/or mitigation toolkit needs to be tailored to the particular bias of interest.  More specifically, it needs to know the attribute or attributes, called _protected attributes_, that are of interest: race is one example of a _protected attribute_ and age is a second.

### The Machine Learning Workflow
To understand how bias can enter a machine learning model, we first review the basics of how a model is created in a supervised machine learning process.  



![image](images/Complex_NoProc_V3.jpg)








First, the process starts with a _training dataset_, which contains a sequence of instances, where each instance has two components: the features and the correct prediction for those features.  Next, a machine learning algorithm is trained on this training dataset to produce a machine learning model.  This generated model can be used to make a prediction when given a new instance.  A second dataset with features and correct predictions, called a _test dataset_, is used to assess the accuracy of the model.
Since this test dataset is the same format as the training dataset, a set of instances of features and prediction pairs, often these two datasets derive from the same initial dataset.  A random partitioning algorithm is used to split the initial dataset into training and test datasets.

Bias can enter the system in any of the three steps above.  The training data set may be biased in that its outcomes may be biased towards particular kinds of instances.  The algorithm that creates the model may be biased in that it may generate models that are weighted towards particular features in the input. The test data set may be biased in that it has expectations on correct answers that may be biased.  These three points in the machine learning process represent points for testing and mitigating bias.  In AI Fairness 360 codebase, we call these points _pre-processing_, _in-processing_, and _post-processing_. 

### AI Fairness 360
We are now ready to utilize AI Fairness 360 (`aif360`) to detect and mitigate bias.  We will use the German credit dataset, splitting it into a training and test dataset.  We will look for bias in the creation of a machine learning model to predict if an applicant should be given credit based on various features from a typical credit application.  The protected attribute will be "Age", with "1" (older than or equal to 25) and "0" (younger than 25) being the values for the privileged and unprivileged groups, respectively.
For this first tutorial, we will check for bias in the initial training data, mitigate the bias, and recheck.  More sophisticated machine learning workflows are given in the author tutorials and demo notebooks in the codebase.

Here are the steps involved
#### Step 1: Write import statements
#### Step 2: Set bias detection options, load dataset, and split between train and test
#### Step 3: Compute fairness metric on original training dataset
#### Step 4: Mitigate bias by transforming the original dataset
#### Step 5: Compute fairness metric on transformed training dataset

### Step 1 Import Statements
As with any python program, the first step will be to import the necessary packages.  Below we import several components from the `aif360` package.  We import the GermanDataset, metrics to check for bias, and classes related to the algorithm we will use to mitigate bias.

In [1]:
# Load all necessary packages
import sys
sys.path.insert(1, "../")  

import numpy as np
np.random.seed(0)
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from aif360.metrics import BinaryLabelDatasetMetric,DatasetMetric
from aif360.algorithms.preprocessing import Reweighing

from IPython.display import Markdown, display

pip install 'aif360[LawSchoolGPA]'


### Step 2 Load dataset, specifying protected attribute, and split dataset into train and test
In Step 2 we load the initial dataset, setting the protected attribute to be age.  We then splits the original dataset into training and testing datasets.  Although we will use only  the training dataset in this tutorial, a normal workflow would also use a test dataset for assessing the efficacy (accuracy, fairness, etc.) during the development of a machine learning model.  Finally, we set two variables (to be used in Step 3) for the privileged (1) and unprivileged (0) values for the age attribute.  These are key inputs for detecting and mitigating bias, which will be Step 3 and Step 4.  

In [2]:
score = pd.read_csv('C:/Users/vivekananda214749/Documents/Bias/EXL_AI_HC_00060/Server_data_new_with_score.csv') 
survey = pd.read_csv('C:/Users/vivekananda214749/Documents/Bias/EXL_AI_HC_00060/Server_data_new_V4.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3325: DtypeWarning: Columns (66,70,71,72,73,75,76,77,78,80,81,82,84,85,90,91,92,93,95,97,98,99,100,101,102,103,104,105,106,107,108,109,110,112,114,115,116,117,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,155,156,158,159,160,161,162,163,164,166,167,168,169,170,171,172,174,176,178,180,189,190,191,195,197,198,199,200,205,206,207,208,218,219,220,221,222,223,224,225,226,227,228,236,238,240,242,244,246,248,250,252,253,255,258,260,261,262,263,264,265,266,267,268,269,270,273,275,278,280,281,284,285,286,287,288,289,290,305,306,307,315,316,318,319,320,321,322,324,325,326,327,328,330,335,336,341,343,348,360,361,362,364) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3325: Dtype

In [3]:
survey['DEDATTN'] = np.where(survey['DEDATTN'] == 5, 4, survey['DEDATTN'])

In [4]:
survey = pd.merge(survey,score[['NUMID','score_sum']],on='NUMID', how='inner')

In [5]:
x_variables = ['DSEX', 'DCLANGS', 'DEDATTN', 'DBQ2430', 'DRACE','DAGE', 'score_sum']
survey = survey[x_variables]
survey.columns

Index(['DSEX', 'DCLANGS', 'DEDATTN', 'DBQ2430', 'DRACE', 'DAGE', 'score_sum'], dtype='object')

In [6]:
survey.head()

,DSEX,DCLANGS,DEDATTN,DBQ2430,DRACE,DAGE,score_sum
0,0,1,2,6,4,2,6
1,0,1,2,6,1,2,10
2,0,1,1,6,2,2,4
3,1,1,2,5,1,2,5
4,0,1,2,3,1,2,6


In [7]:
ohe_columns = ['DCLANGS', 'DEDATTN', 'DBQ2430', 'DRACE','DAGE']
survey_ohe = pd.get_dummies(survey, columns = ohe_columns, prefix_sep = '_', drop_first = True)
survey_ohe.head()

,DSEX,score_sum,DCLANGS_2,DCLANGS_3,DEDATTN_2,DEDATTN_3,DEDATTN_4,DBQ2430_2,DBQ2430_3,DBQ2430_4,...,DBQ2430_7,DBQ2430_8,DRACE_2,DRACE_3,DRACE_4,DAGE_2,DAGE_3,DAGE_4,DAGE_5,DAGE_6
0,0,6,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
1,0,10,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,4,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3,1,5,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,6,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [8]:
#skip
data_scaled_df = survey_ohe.copy()
scale_columns = ['score_sum']
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled_df['score_sum_scaled'] = pd.DataFrame(scaler.fit_transform(survey_ohe['score_sum'].values.reshape(-1,1)))

In [9]:
data_scaled_df['score_binary']=np.where(data_scaled_df['score_sum_scaled']<=0.4,1,0)

In [10]:
data_scaled_df.shape

(16384, 24)

In [11]:
data_scaled_df.isnull().sum()

DSEX                0
score_sum           0
DCLANGS_2           0
DCLANGS_3           0
DEDATTN_2           0
DEDATTN_3           0
DEDATTN_4           0
DBQ2430_2           0
DBQ2430_3           0
DBQ2430_4           0
DBQ2430_5           0
DBQ2430_6           0
DBQ2430_7           0
DBQ2430_8           0
DRACE_2             0
DRACE_3             0
DRACE_4             0
DAGE_2              0
DAGE_3              0
DAGE_4              0
DAGE_5              0
DAGE_6              0
score_sum_scaled    0
score_binary        0
dtype: int64

In [12]:
data_scaled_df['score_binary'].value_counts()

0    13961
1     2423
Name: score_binary, dtype: int64

In [13]:
x_independent = data_scaled_df.loc[:, (data_scaled_df.columns != 'score_sum') & (data_scaled_df.columns != 'score_sum_scaled')]
y_dependent = data_scaled_df[['score_sum_scaled']]

In [14]:
from aif360.datasets import BinaryLabelDataset

data_scaled_df = BinaryLabelDataset(
    protected_attribute_names=['DSEX'],
    favorable_label=1, unfavorable_label=0,
    label_names=['score_binary'],                                             
    df=data_scaled_df,                                             
         
     # ignore sex-related attributes
)


In [15]:
dataset_orig_train,dataset_orig_test = data_scaled_df.split([0.7], shuffle=True)


privileged_groups =   [{'DSEX': 1}]
unprivileged_groups = [{'DSEX': 0}]

### Step 3 Compute fairness metric on original training dataset
Now that we've identified the protected attribute 'age' and defined privileged and unprivileged values, we can use aif360 to detect bias in the dataset.  One simple test is to compare the percentage of favorable results for the privileged and unprivileged groups, subtracting the former percentage from the latter.   A negative value indicates less favorable outcomes for the unprivileged groups.  This is implemented in the method called mean_difference on the BinaryLabelDatasetMetric class.  The code below performs this check and displays the output, showing that the difference is -0.169905.

In [16]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.035927


### Step 4 Mitigate bias by transforming the original dataset
The previous step showed that the privileged group was getting 17% more positive outcomes in the training dataset.   Since this is not desirable, we are going to try to mitigate this bias in the training dataset.  As stated above, this is called _pre-processing_ mitigation because it happens before the creation of the model.  

AI Fairness 360 implements several pre-processing mitigation algorithms.  We will choose the Reweighing algorithm [1], which is implemented in the `Reweighing` class in the `aif360.algorithms.preprocessing` package.  This algorithm will transform the dataset to have more equity in positive outcomes on the protected attribute for the privileged and unprivileged groups.

We then call the fit and transform methods to perform the transformation, producing a newly transformed training dataset (dataset_transf_train).

`[1] F. Kamiran and T. Calders,  "Data Preprocessing Techniques for Classification without Discrimination," Knowledge and Information Systems, 2012.`

In [17]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

### Step 5 Compute fairness metric on transformed dataset
Now that we have a transformed dataset, we can check how effective it was in removing bias by using the same metric we used for the original training dataset in Step 3.  Once again, we use the function mean_difference in the BinaryLabelDatasetMetric class.   We see the mitigation step was very effective, the difference in mean outcomes is now 0.0.  So we went from a 17% advantage for the privileged group to equality in terms of mean outcome.

In [18]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


### Summary
The purpose of this tutorial is to give a new user to bias detection and mitigation a gentle introduction to some of the functionality of AI Fairness 360.  A more complete use case would take the next step and see how the transformed dataset impacts the accuracy and fairness of a trained model.  This is implemented in the demo notebook in the examples directory of toolkit, called demo_reweighing_preproc.ipynb.  I highly encourage readers to view that notebook as it is  generalization and extension of this simple tutorial.

There are many metrics one can use to detect the presence of bias. AI Fairness 360 provides many of them for your use. Since it is not clear which of these metrics to use, we also provide some guidance. Likewise, there are many different bias mitigation algorithms one can employ, many of which are in AI Fairness 360. Other tutorials will demonstrate the use of some of these metrics and mitigations algorithms.

As mentioned earlier, both fairness metrics and mitigation algorithms can be performed at various stages of the machine learning pipeline.  We recommend checking for bias as often as possible, using as many metrics are relevant for the application domain.  We also recommend incorporating bias detection in an automated continuous integration pipeline to ensure bias awareness as a software project evolves.